# GDS Taper
> Create a Taper and Simulate

In [3]:
import meow as mw
import gdsfactory as gf  # pip install meow-sim[gds]
from gdsfactory.cross_section import cross_section
import matplotlib.pyplot as plt
import numpy as np

## Example Taper

> Note: `meow` expects the propagation direction to be the `z`-axis! This makes the `zx`-plane parallel with the chip and the `y`-axis perpendicular to the chip. Somewhat confusingly, the (x, y) GDS coordinate tuple hence maps onto the (z, x) `meow` coordinate tuple. Whereas the y coordinate from meow denotes the direction perpendicular to the chip. (I will probably change the `meow` convention in the future.)

In [4]:
def example_gds_cross_section(
    width: float = 0.450,
    clad_width: float = 2.0,
) -> gf.CrossSection:
    """a strip waveguide cross section

    Args:
        width:  the width of the strip waveguide
        clad_width: the width of the cladding
    """
    core_width = width
    port_names = ("in0", "out0")
    sections = (
        gf.Section(width=core_width, offset=0, layer=(1, 0), name="core"),
        gf.Section(
            width=clad_width,
            offset=0.5 * (core_width + clad_width),
            layer=(2, 0),
            name="upper",
        ),
        gf.Section(
            width=clad_width,
            offset=-0.5 * (core_width + clad_width),
            layer=(2, 0),
            name="lower",
        ),
    )
    cs = cross_section(
        width=width,
        port_names=port_names,
        sections=sections,
    )
    return cs

In [5]:
@gf.cell
def example_taper(
    width_input: float = 0.450,
    width_output: float = 1.0,
    length: float = 10.0,
) -> gf.Component:
    """create a linear taper

    Args:
        width_input: input width of the linear taper
        width_output: output width of the linear taper
        length: the length of the linear taper
    """
    input_cs = example_gds_cross_section(width_input)
    output_cs = example_gds_cross_section(width_output)
    transition = gf.path.transition(input_cs, output_cs, width_type="linear")
    length = gf.snap.snap_to_grid(length)  # type: ignore
    path = gf.path.straight(length)
    component = gf.path.extrude(path, transition)
    return component

In [6]:
taper = example_taper(width_input=0.45, width_output=1.0, length=20)
taper

2024-06-11 11:26:41.841 | ERROR    | kfactory.kcell:wrapper_autocell:3323 - An error has been caught in function 'wrapper_autocell', process 'MainProcess' (351433), thread 'MainThread' (139980713010240):
Traceback (most recent call last):

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/archie/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
    │   └ <bound method Application.launch_instance of <class 'ipykernel.kernelapp.IPKernelApp'>>
    └ <module 'ipykernel.kernelapp' from '/home/archie/miniconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py'>
  File "/home/archie/miniconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
    │   └ <function IPKernelApp.start at 0x7f4fc93acf40>
    └ <ipykernel.kernelapp.IPKernelApp object at 0x7f4fcc49ce90>
  File "/home/archie/miniconda3/lib/py

TypeError: cross_section() got an unexpected keyword argument 'auto_widen'

## Example Structure Map

In [ ]:
def example_extrusions(
    t_slab: float = 0.020,
    t_soi: float = 0.220,
    t_ox: float = 1.0,
):
    """create some simple extrusion rules

    Args:
        t_slab: the slab thickness
        t_soi: the SOI thickness
        t_ox: the oxide layer thickness
    """
    extrusions = {
        (1, 0): [
            mw.GdsExtrusionRule(
                material=mw.silicon,
                h_min=0.0,
                h_max=0.0 + t_soi,
                mesh_order=1,
            ),
            mw.GdsExtrusionRule(
                material=mw.silicon_oxide,
                h_min=-1.0,
                h_max=t_soi + t_ox,
                buffer=t_ox / 2,
                mesh_order=2,
            ),
        ],
        (2, 0): [
            mw.GdsExtrusionRule(
                material=mw.silicon,
                h_min=0.0,
                h_max=0.0 + t_slab,
                mesh_order=1,
            ),
            mw.GdsExtrusionRule(
                material=mw.silicon_oxide,
                h_min=-1.0,
                h_max=t_slab + t_ox,
                mesh_order=2,
            ),
        ],
    }
    return extrusions

## Extrude GDS

In [ ]:
extrusion_rules = example_extrusions()
structs = mw.extrude_gds(taper, extrusion_rules)
mw.visualize(structs, scale=(1, 1, 0.2))

## Divide into Cells

In [ ]:
w_sim = 1.0
h_sim = 1.0
mesh = 100
num_cells = 10
taper_length = np.diff(taper.bbox[:, 0]).item()
Ls = [taper_length / num_cells for _ in range(num_cells)]
print(Ls)

cells = mw.create_cells(
    structures=structs,
    mesh=mw.Mesh2d(
        x=np.linspace(-0.75, 0.75, mesh + 1),
        y=np.linspace(-0.3, 0.5, mesh + 1),
    ),
    Ls=Ls,
)

mw.visualize(cells[0], cbar=False)
plt.show()

mw.visualize(cells[-1], cbar=False)
plt.show()

## Find Cross Sections

In [ ]:
env = mw.Environment(wl=1.55, T=25.0)
css = [mw.CrossSection.from_cell(cell=cell, env=env) for cell in cells]

mw.visualize(css[0])
mw.visualize(css[-1])

In [ ]:
num_modes = 4
modes = mw.compute_modes(css[0], num_modes=num_modes)
mw.visualize(modes[0])

## Compute Modes (FDE)

In [ ]:
%%time
num_modes = 4
modes = [mw.compute_modes(cs, num_modes=num_modes) for cs in css]

In [ ]:
mw.visualize(modes[0][0], fields=["Hx"])
mw.visualize(modes[-1][1], fields=["Hx"])

## Calculate S-matrix (EME)

In [ ]:
S, port_map = mw.compute_s_matrix(modes, cells)
print(port_map)
mw.visualize((abs(S), port_map))